In [5]:
%pip install -q Faker==25.8.0

Note: you may need to restart the kernel to use updated packages.


## Configuracao do ambiente e reprodutibilidade


In [7]:

from pathlib import Path
import numpy as np
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == 'Parte01':
    ROOT = ROOT.parent

SEED = 42
rng = np.random.default_rng(SEED)

print('Vers�es -> pandas:', pd.__version__, '| numpy:', np.__version__)


Vers�es -> pandas: 2.3.2 | numpy: 2.2.6


## Geracao de dados sinteticos de vendas


In [ ]:
from datetime import datetime, timedelta
from faker import Faker

EXTENDER_ATE_2024_06 = False
N_LINHAS = 300
START_2023 = "2023-01-01"
END_2023   = "2023-12-31"
END_DATE   = "2024-06-30" if EXTENDER_ATE_2024_06 else END_2023



CATEGORIAS = {
    "Eletrônicos": ["Fone Pro", "Mouse Óptico", "Teclado Mecânico", "Smartwatch", "Caixa Bluetooth"],
    "Casa & Cozinha": ["Liquidificador Turbo", "Panela Antiaderente", "Aspirador 900", "Jogo de Facas", "Cafeteira X"],
    "Esporte": ["Bola Oficial", "Tênis Running", "Garrafa Térmica", "Mochila Fitness", "Camiseta Dry"],
    "Moda": ["Camiseta Básica", "Calça Jeans", "Jaqueta Leve", "Tênis Casual", "Boné Snapback"],
    "Brinquedos": ["Quebra-cabeça 500pç", "Carrinho RC", "Boneca Classic", "Blocos Criativos", "Jogo de Tabuleiro"],
    "Material-Escolar": ["Caderno", "lapis", "Caneta", "MOchila", "estojo"]
}

PRECO_PARAMS = {
    "Eletrônicos":  {"mu": np.log(300), "sigma": 0.6, "minv": 50},
    "Casa & Cozinha":{"mu": np.log(150), "sigma": 0.5, "minv": 30},
    "Esporte":      {"mu": np.log(120), "sigma": 0.5, "minv": 20},
    "Moda":         {"mu": np.log(90),  "sigma": 0.5, "minv": 20},
    "Brinquedos":   {"mu": np.log(80),  "sigma": 0.6, "minv": 15},
    "Material-Escolar": {"mu": np.log(90), "sigma": 0.5, "minv": 20}
}

SAZONALIDADE = {
    1: 1.05, 2: 1.15, 3: 0.98, 4: 1.00,
    5: 1.02, 6: 0.98, 7: 0.95, 8: 1.10,
    9: 1.00, 10: 1.05, 11: 1.30, 12: 1.35,

}

SENS_CAT = {
    "Material-Escolar": {1: 1.10, 2: 1.35, 7: 1.10, 8: 1.25},
    "Brinquedos": {10: 1.10, 11: 1.25, 12: 1.40},
    "Moda": {5: 1.05, 6: 1.08, 11: 1.10, 12: 1.10},
    "Eletrônicos":{11: 1.35, 12: 1.10},

    
}


def gerar_datas(n, start_date: str, end_date: str, rng: np.random.Generator):
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    delta_dias = (end - start).days + 1 
    offs = rng.integers(0, delta_dias, size=n)
    return np.array([start + timedelta(days=int(o)) for o in offs])

def preco_por_categoria(cat: str, rng: np.random.Generator) -> float:
    p = PRECO_PARAMS[cat]
    val = rng.lognormal(mean=p["mu"], sigma=p["sigma"])
    val = max(val, p["minv"])
    return round(float(val), 2)


fake = Faker("pt_BR")
fake.seed_instance(SEED)
n = max(N_LINHAS, 50)

categorias = rng.choice(list(CATEGORIAS.keys()), size=n)
produtos   = np.array([rng.choice(CATEGORIAS[c]) for c in categorias])
datas      = gerar_datas(n, START_2023, END_DATE, rng)
fatores_mes = np.array([SAZONALIDADE.get(d.month, 1.0) for d in datas])


ajuste_cat = np.ones(n)
for i, (cat, d) in enumerate(zip(categorias, datas)):
    ajuste_cat[i] = SENS_CAT.get(cat, {}).get(d.month, 1.0)

# --- Quantidade (Poisson) com clipping saudável ---
lambda_base = 3.0
lam = lambda_base * fatores_mes * ajuste_cat
quantidade = rng.poisson(lam=lam, size=n) + 1
quantidade = np.clip(quantidade, 1, 12)  # ninguém leva 80 unidades num ticket normal

# --- Preços por categoria ---
precos = np.array([preco_por_categoria(cat, rng) for cat in categorias])

# --- IDs e DataFrame ---
ids = np.arange(1, n + 1)
df = pd.DataFrame({
    "ID": ids,
    "Data": pd.to_datetime(datas).strftime("%Y-%m-%d"),
    "Produto": produtos,
    "Categoria": categorias,
    "Quantidade": quantidade,
    "Preco": precos,
})

# --- Salvar bruto ---
raw_path = ROOT / "data" / "raw" / "vendas_2023.csv"
raw_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(raw_path, index=False, encoding="utf-8")

print(f"CSV bruto salvo em: {raw_path}")
print("Linhas totais (inclui duplicatas intencionais):", len(df))
display(df.head())

# --- Checagem rápida de realismo: total mensal e variação m/m ---
df_tmp = df.copy()
df_tmp["Data"] = pd.to_datetime(df_tmp["Data"])
df_tmp["Total"] = df_tmp["Quantidade"] * df_tmp["Preco"]
mensal = df_tmp.groupby(df_tmp["Data"].dt.to_period("M"))["Total"].sum().astype(float)
mm = mensal.to_timestamp()
var_mm = (mm.pct_change() * 100).round(1)
print("\nTotal mensal (R$) e variação m/m (%)")
display(pd.DataFrame({"Total": mm.round(2), "m/m %": var_mm}))


CSV bruto salvo em: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv
Linhas totais (inclui duplicatas intencionais): 300


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,3,237.60
1,2,2023-10-12,Jogo de Tabuleiro,Brinquedos,2,194.12
2,3,2023-09-27,Camiseta Básica,Moda,5,115.94
3,4,2023-02-19,Camiseta Dry,Esporte,4,62.28
4,5,2023-10-29,Garrafa Térmica,Esporte,5,167.86



Total mensal (R$) e variação m/m (%)


,Total,m/m %
Data,,
2023-01-01,20652.24,NaN
2023-02-01,21878.71,5.9
2023-03-01,13038.91,-40.4
2023-04-01,17146.39,31.5
2023-05-01,10832.60,-36.8
2023-06-01,13924.77,28.5
2023-07-01,12878.85,-7.5
2023-08-01,18386.97,42.8
2023-09-01,16332.89,-11.2


## Leitura do CSV bruto gerado


In [149]:
raw_path = ROOT / "data/raw/vendas_2023.csv"
df = pd.read_csv(raw_path)

## Simulacao de inconsistencias e duplicatas


In [150]:
n = len(df)

if "rgn" not in globals():
    SEED = 42
    rng = np.random.default_rng(SEED)
r = rng.random

rates = {
    "categoria_nan":        0.15,  # 15% Categoria ausente
    "produto_nan":          0.07,  # 7%  Produto ausente
    "preco_nan":            0.12,  # 12% Preco ausente
    "quantidade_nan":       0.08,  # 8%  Quantidade ausente
    "quantidade_str":       0.12,  # 12% Quantidade vira string
    "quantidade_zero_neg":  0.03,  # 4%  Quantidade vira 0/-1 (inválido)
    "data_nan":             0.03,  # 5%  Data ausente
    "data_invalida":        0.01,  # 4%  Data inválida (strings que quebram o parser)
    "dup_linha":            0.05,  # 5%  duplicatas idênticas
    "dup_chave":            0.05,  # 5%  duplicatas por (ID,Data) com valores diferentes
}

m = r(n) < rates["categoria_nan"];  df.loc[m, "Categoria"] = None
m = r(n) < rates["produto_nan"];    df.loc[m, "Produto"] = None
m = r(n) < rates["preco_nan"];      df.loc[m, "Preco"] = None
m = r(n) < rates["quantidade_nan"]; df.loc[m, "Quantidade"] = None
m = r(n) < rates["data_nan"];       df.loc[m, "Data"] = None   

m = r(n) < rates["quantidade_str"]
df.loc[m, "Quantidade"] = (
    pd.to_numeric(df.loc[m, "Quantidade"], errors= "coerce")
      .fillna(0).astype(int).astype(str)
)

m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "2023-13-01"
m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "31/02/2023"

k = max(1, int(rates["dup_chave"] * n))
idx = rng.choice(df.index, size=k, replace=False)
dups = df.loc[idx].copy()

dups["Preco"] = pd.to_numeric(dups["Preco"], errors="coerce") * (1 + rng.normal(0, 0.05, size=len(dups)))
dups["Quantidade"] = (
    pd.to_numeric(dups["Quantidade"], errors="coerce").fillna(1).astype(int)
    + rng.integers(-1, 2, size=len(dups))
)

df = pd.concat([df, dups], ignore_index=True)
df = df.sort_values(by=['ID'], kind='mergesort').reset_index(drop=True)
df.to_csv(raw_path, index=False, encoding="utf-8")
print(f"[OK] Sujeiras aplicadas e salvo: {raw_path} | linhas={len(df)}")

[OK] Sujeiras aplicadas e salvo: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv | linhas=315


C:\Users\crish\AppData\Local\Temp\ipykernel_22980\1568022500.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['0' '2' '3' '3' '3' '4' '3' '6' '3' '4' '2' '0' '4' '2' '7' '3' '2' '2'
 '6' '5' '3' '5' '6']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[m, "Quantidade"] = (


## Diagnostico exploratorio do CSV com problemas


In [151]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

def diagnosticar_csv_plus(
    path_csv,
    keys_opcionais=None,                 # ex.: [("ID","Data"), ("Produto","Data")]
    periodo=("2023-01-01","2024-06-30"), # janela válida para Data
    mostrar_head=3,
    sample_k=3                           # quantos exemplos de linhas problemáticas mostrar
):
    df = pd.read_csv(path_csv)
    nlin, ncol = df.shape
    print(f"Arquivo: {path_csv} | shape = ({nlin}, {ncol})\n")

    # 1) Head e dtypes
    if mostrar_head:
        print(f"Amostra (head {mostrar_head}):")
        display(df.head(mostrar_head))
    print("\nTipos por coluna:")
    display(df.dtypes.to_frame("dtype"))

    # 2) Nulos (contagem e %)
    print("\nValores ausentes por coluna:")
    nulos = df.isna().sum()
    pct   = (nulos / max(len(df),1) * 100).round(2)
    out_nulos = (pd.DataFrame({"nulos": nulos, "pct_%": pct})
                   .sort_values("nulos", ascending=False))
    display(out_nulos)

    # 3) Duplicatas (linha inteira + por chaves)
    print("\nDuplicatas (linha inteira):", int(df.duplicated().sum()))
    if keys_opcionais:
        rows = []
        # normaliza: aceitar ("ID","Data") ou [("ID","Data"), ...]
        if isinstance(keys_opcionais, (tuple, list)) and all(isinstance(x, str) for x in keys_opcionais):
            keys_opcionais = [tuple(keys_opcionais)]
        for ch in keys_opcionais:
            ch = tuple(ch)
            rows.append({"chaves": str(ch),
                         "duplicatas": int(df.duplicated(subset=list(ch)).sum())})
        print("\nDuplicatas por chave:")
        display(pd.DataFrame(rows).sort_values("duplicatas", ascending=False, ignore_index=True))

    # 4) Texto que parece número (tipo misto)
    print("\nColunas 'object' que parecem numéricas:")
    rows = []
    for col in df.columns:
        if df[col].dtype == "object":
            conv = pd.to_numeric(df[col], errors="coerce")
            mask = df[col].notna() & conv.notna()
            qtd  = int(mask.sum())
            if qtd > 0:
                exemplos = (df.loc[mask, col].astype(str).drop_duplicates()[:sample_k].tolist())
                rows.append({"coluna": col, "qtd_texto_numerico": qtd, "pct_%": round(qtd/len(df)*100,2),
                             "exemplos": exemplos})
    if rows:
        display(pd.DataFrame(rows).sort_values("qtd_texto_numerico", ascending=False, ignore_index=True))
    else:
        print("  (nenhuma)")

    # 5) Problemas específicos por coluna

    # 5.1 Data: inválida e fora do período
    if "Data" in df.columns:
        dt = pd.to_datetime(df["Data"], errors="coerce")
        mask_invalida = dt.isna() & df["Data"].notna()     # strings inválidas (não conta NaN já acima)
        inicio, fim = pd.to_datetime(periodo[0]), pd.to_datetime(periodo[1])
        mask_fora = dt.notna() & ((dt < inicio) | (dt > fim))
        print("\n[Data] inválida:", int(mask_invalida.sum()), " | fora do período:", int(mask_fora.sum()))
        if mask_invalida.any():
            print("Exemplos Data inválida:")
            display(df.loc[mask_invalida].head(sample_k))
        if mask_fora.any():
            print("Exemplos Data fora do período:")
            display(df.loc[mask_fora].head(sample_k))

    # 5.2 Quantidade: não numérica, zeros/negativos
    if "Quantidade" in df.columns:
        q_conv = pd.to_numeric(df["Quantidade"], errors="coerce")
        mask_nao_num = df["Quantidade"].notna() & q_conv.isna()
        mask_zero_neg = q_conv.notna() & (q_conv <= 0)
        print("\n[Quantidade] não numérica:", int(mask_nao_num.sum()), " | <= 0:", int(mask_zero_neg.sum()))
        if mask_nao_num.any():
            print("Exemplos Quantidade não numérica:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_zero_neg.any():
            print("Exemplos Quantidade <= 0:")
            display(df.loc[mask_zero_neg].head(sample_k))

    # 5.3 Preco: não numérico, <= 0
    if "Preco" in df.columns:
        p_conv = pd.to_numeric(df["Preco"], errors="coerce")
        mask_nao_num = df["Preco"].notna() & p_conv.isna()
        mask_le_zero = p_conv.notna() & (p_conv <= 0)
        print("\n[Preco] não numérico:", int(mask_nao_num.sum()), " | <= 0:", int(mask_le_zero.sum()))
        if mask_nao_num.any():
            print("Exemplos Preco não numérico:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_le_zero.any():
            print("Exemplos Preco <= 0:")
            display(df.loc[mask_le_zero].head(sample_k))

    return df  # permite encadear outras análises

resultado = diagnosticar_csv_plus(raw_path, keys_opcionais=[("ID","Data")])
resultado.head()



Arquivo: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv | shape = (315, 6)

Amostra (head 3):


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,3.0,237.60
1,2,2023-10-12,Jogo de Tabuleiro,Brinquedos,2.0,194.12
2,3,2023-09-27,Camiseta Básica,Moda,NaN,115.94



Tipos por coluna:


,dtype
ID,int64
Data,object
Produto,object
Categoria,object
Quantidade,float64
Preco,float64



Valores ausentes por coluna:


...

,ID,Data,Produto,Categoria,Quantidade,Preco
36,37,2023-13-01,estojo,NaN,6.0,45.90
207,199,31/02/2023,Camiseta Dry,Esporte,5.0,85.53
298,284,31/02/2023,Bola Oficial,Esporte,7.0,115.88



[Quantidade] não numérica: 0  | <= 0: 2
Exemplos Quantidade <= 0:


,ID,Data,Produto,Categoria,Quantidade,Preco
16,17,2023-04-02,Tênis Casual,Moda,0.0,211.62
175,169,2023-02-21,Bola Oficial,Esporte,0.0,NaN



[Preco] não numérico: 0  | <= 0: 0


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,3.0,237.60
1,2,2023-10-12,Jogo de Tabuleiro,Brinquedos,2.0,194.12
2,3,2023-09-27,Camiseta Básica,Moda,NaN,115.94
3,4,2023-02-19,Camiseta Dry,Esporte,4.0,62.28
4,5,NaN,Garrafa Térmica,NaN,5.0,167.86


## Limpeza e padronizacao das vendas


In [157]:
from pathlib import Path
from typing import Dict, Tuple
import numpy as np
import pandas as pd


COLS = ["ID", "Data", "Produto", "Categoria", "Quantidade", "Preco"]

_base_root = globals().get("ROOT", Path.cwd().resolve())
if isinstance(_base_root, Path) and _base_root.name == "Parte01":
    _base_root = _base_root.parent

_data_dir = globals().get("DATA_DIR", _base_root / "data")
_raw_dir = globals().get("RAW_DIR", _data_dir / "raw")
_processed_dir = globals().get("PROCESSED_DIR", _data_dir / "processed")

_raw_dir.mkdir(parents=True, exist_ok=True)
_processed_dir.mkdir(parents=True, exist_ok=True)

RAW_PATH = (_raw_dir / "vendas_2023.csv").resolve()
CLEAN_PATH = (_processed_dir / "data_clean.csv").resolve()

print("RAW_PATH  :", RAW_PATH)
print("CLEAN_PATH:", CLEAN_PATH)

# ---------------------------
# Helpers
# ---------------------------
def _first_non_null(s: pd.Series):
    s2 = s.dropna()
    return s2.iloc[0] if not s2.empty else pd.NA

def _mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if s2.empty:
        return pd.NA
    mode = s2.mode()
    return mode.iloc[0] if not mode.empty else s2.iloc[0]

def _to_num(s: pd.Series):
    s_clean = s.astype(str).str.strip()
    s_clean = s_clean.str.replace('\xa0', '', regex=False)
    mask_comma = s_clean.str.contains(',', regex=False)
    s_clean = s_clean.where(~mask_comma, s_clean.str.replace('.', '', regex=False))
    s_clean = s_clean.str.replace(',', '.', regex=False)
    return pd.to_numeric(s_clean, errors='coerce')

def _stringify(s: pd.Series):
    return (
        s.astype(str)
         .str.strip()
         .replace({'': pd.NA, 'nan': pd.NA})
    )

def _maybe_to_datetime(s: pd.Series):
    return pd.to_datetime(s, errors='coerce', format='mixed')

def clean_sales_csv(
    input_path: Path,
    *,
    output_path: Path | None = None,
    key_cols=("ID", "Data"),
    price_decimals: int = 2,
    save_float_format: str = "%.2f",
) -> Tuple[pd.DataFrame, Dict[str, int]]:
    input_path = Path(input_path)
    if output_path is None:
        output_path = CLEAN_PATH
    else:
        output_path = Path(output_path)

    report: Dict[str, int] = {}

    if not input_path.exists():
        raise FileNotFoundError(f"Arquivo de entrada n???o encontrado: {input_path}")

    output_path.parent.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(input_path)
    df = df.copy()

    missing_cols = [c for c in COLS if c not in df.columns]
    if missing_cols:
        raise ValueError(f"Colunas esperadas ausentes: {missing_cols}")

    df = df[COLS].copy()
    report["linhas_iniciais"] = int(len(df))

    df["Data"] = _maybe_to_datetime(df["Data"])
    report["datas_invalidas"] = int(df["Data"].isna().sum())

    df["Produto"] = _stringify(df["Produto"]).str.replace(r"\s+", " ", regex=True).str.strip()
    df["Categoria"] = _stringify(df["Categoria"]).str.replace(r"\s+", " ", regex=True).str.strip()

    # Padroniza capitaliza????o b??sica (mant??m NaNs para imputa????o pontual)
    df["Produto"] = df["Produto"].str.title()
    df["Categoria"] = df["Categoria"].str.title()

    duplicates_exact = int(df.duplicated(keep='first').sum())
    report["duplicatas_exatas"] = duplicates_exact
    df = df.drop_duplicates(keep='first').reset_index(drop=True)

    dup_id = df.duplicated(subset=["ID"], keep='first')
    report["ids_duplicados"] = int(dup_id.sum())
    df = df.loc[~dup_id].copy()

    prod_mode_by_cat = (
        df.groupby("Categoria", dropna=False)["Produto"].agg(_mode_or_first)
    )
    cat_mode_by_prod = (
        df.groupby("Produto", dropna=False)["Categoria"].agg(_mode_or_first)
    )

    # S?? preenche valores ausentes; mant??m produto/categoria originais distintos
    df["Produto"] = df["Produto"].fillna(df["Categoria"].map(prod_mode_by_cat))
    df["Categoria"] = df["Categoria"].fillna(df["Produto"].map(cat_mode_by_prod))

    df["Quantidade"] = _to_num(df["Quantidade"])
    df["Preco"] = _to_num(df["Preco"])

    q_nonpos = int((df["Quantidade"] <= 0).fillna(False).sum())
    p_nonpos = int((df["Preco"] <= 0).fillna(False).sum())
    df.loc[df["Quantidade"] <= 0, "Quantidade"] = np.nan
    df.loc[df["Preco"] <= 0, "Preco"] = np.nan
    report["quantidade_nao_positiva_para_NaN"] = q_nonpos
    report["preco_nao_positivo_para_NaN"] = p_nonpos

    med_preco_prod = df.groupby("Produto", dropna=False)["Preco"].median()
    med_qtd_prod = df.groupby("Produto", dropna=False)["Quantidade"].median()

    p_missing_before = int(df["Preco"].isna().sum())
    df["Preco"] = df["Preco"].fillna(df["Produto"].map(med_preco_prod))
    df["Preco"] = df["Preco"].fillna(df["Preco"].median())
    report["preco_imputado"] = p_missing_before - int(df["Preco"].isna().sum())

    q_missing_before = int(df["Quantidade"].isna().sum())
    df["Quantidade"] = df["Quantidade"].fillna(df["Produto"].map(med_qtd_prod))
    df["Quantidade"] = df["Quantidade"].fillna(1)
    report["quantidade_imputada"] = q_missing_before - int(df["Quantidade"].isna().sum())

    dup_key_count = int(df.duplicated(subset=list(key_cols)).sum())
    report["duplicatas_por_chave_encontradas"] = dup_key_count

    def _agg_quantidade(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    def _agg_preco(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    df = (
        df.groupby(list(key_cols), as_index=False)
          .agg({
              "Produto": _mode_or_first,
              "Categoria": _mode_or_first,
              "Quantidade": _agg_quantidade,
              "Preco": _agg_preco
          })
    )

    prod_missing_before = int(df['Produto'].isna().sum())
    cat_missing_before = int(df['Categoria'].isna().sum())

    prod_to_cat = (
        df[['Produto', 'Categoria']].dropna().drop_duplicates()
          .groupby('Produto')['Categoria']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Categoria'] = df['Categoria'].fillna(df['Produto'].map(prod_to_cat))

    cat_to_prod = (
        df[['Categoria', 'Produto']].dropna().drop_duplicates()
          .groupby('Categoria')['Produto']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Produto'] = df['Produto'].fillna(df['Categoria'].map(cat_to_prod))

    prod_missing_after = int(df['Produto'].isna().sum())
    cat_missing_after = int(df['Categoria'].isna().sum())
    report['produto_imputado'] = prod_missing_before - prod_missing_after
    report['categoria_imputada'] = cat_missing_before - cat_missing_after

    mask_drop = df['Produto'].isna() | df['Categoria'].isna()
    if mask_drop.any():
        report['linhas_descartadas_sem_produto_ou_categoria'] = int(mask_drop.sum())
        df = df.loc[~mask_drop].copy()

    df["ID"] = df["ID"].astype("int64")
    df["Quantidade"] = pd.Series(df["Quantidade"]).round().astype("Int64")
    df["Preco"] = pd.Series(df["Preco"]).astype("float64").round(price_decimals)

    df["Produto"] = df["Produto"].astype("string")
    df["Categoria"] = df["Categoria"].astype("string")

    df = df.sort_values(by=list(key_cols), kind='mergesort').reset_index(drop=True)
    df.to_csv(output_path, index=False, encoding='utf-8', float_format=save_float_format)

    report["linhas_finais"] = int(len(df))
    report["arquivo_limpo"] = str(output_path)

    return df, report

if __name__ == "__main__":
    df_clean, resumo = clean_sales_csv(RAW_PATH, output_path=CLEAN_PATH)

    df_disk = pd.read_csv(CLEAN_PATH)
    print("len(df_disk) (lido do disco):", len(df_disk))

    print("Resumo da limpeza:")
    for k, v in resumo.items():
        print(f"- {k}: {v}")

    print("Dtypes:")
    print(df_clean.dtypes)

    print("Amostra do Preco (2 casas):")
    print(df_clean["Preco"].head(10))

    total_preview = (df_clean["Quantidade"].fillna(0).astype(float) * df_clean["Preco"]).round(2)
    print("Amostra do Total (2 casas):")
    print(total_preview.head(10))



RAW_PATH  : C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv
CLEAN_PATH: C:\DEV\Teste_Analytics_CristovamPaulo\data\processed\data_clean.csv
len(df_disk) (lido do disco): 291
Resumo da limpeza:
- linhas_iniciais: 315
- datas_invalidas: 9
- duplicatas_exatas: 0
- ids_duplicados: 15
- quantidade_nao_positiva_para_NaN: 2
- preco_nao_positivo_para_NaN: 0
- preco_imputado: 42
- quantidade_imputada: 16
- duplicatas_por_chave_encontradas: 0
- produto_imputado: 2
- categoria_imputada: 0
- linhas_finais: 291
- arquivo_limpo: C:\DEV\Teste_Analytics_CristovamPaulo\data\processed\data_clean.csv
Dtypes:
ID                     int64
Data          datetime64[ns]
Produto       string[python]
Categoria     string[python]
Quantidade             Int64
Preco                float64
dtype: object
Amostra do Preco (2 casas):
0    237.60
1    194.12
2    115.94
3     62.28
4     61.10
5    212.21
6     50.94
7    117.30
8    512.60
9     61.10
Name: Preco, dtype: float64
Amostra do Total (2 casa

## Analise de vendas por produto


In [ ]:
_data_dir = globals().get("DATA_DIR", _base_root / "data")
_processed_dir = globals().get("PROCESSED_DIR", _data_dir / "processed")

OUT_PRODUTOS = (_processed_dir / "vendas_por_produto.csv").resolve()

from IPython.display import display


def analisar_vendas(clean_path=CLEAN_PATH, out_path=OUT_PRODUTOS):
    # 1) Carrega base limpa
    df = pd.read_csv(clean_path)

    # 2) Calcula Total (preferindo centavos inteiros se dispon?veis)
    if "Preco_centavos" in df.columns:
        df["Quantidade"] = pd.to_numeric(df["Quantidade"], errors="coerce").fillna(0).round().astype("Int64")
        df["Preco_centavos"] = pd.to_numeric(df["Preco_centavos"], errors="coerce").fillna(0).round().astype("Int64")
        df["total_cent"] = (df["Quantidade"] * df["Preco_centavos"]).astype("Int64")
        df["Total"] = (df["total_cent"] / 100).astype(float)
    else:
        df["Quantidade"] = pd.to_numeric(df["Quantidade"], errors="coerce").fillna(0).round().astype(int)
        df["Preco"] = pd.to_numeric(df["Preco"], errors="coerce").fillna(0).astype(float)
        df["Total"] = (df["Quantidade"] * df["Preco"]).astype(float)

    # 3) Soma por produto (TODOS os produtos)
    vendas_produto = (
        df.groupby("Produto", as_index=False)["Total"]
          .sum()
          .sort_values("Total", ascending=False)
          .reset_index(drop=True)
    )

    # 4) Salva CSV completo (n?o apenas Top-5)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    vendas_produto.to_csv(out_path, index=False, encoding="utf-8", float_format="%.2f")

    # 5) Relat?rio rápido em console
    print(f"Produtos totais no arquivo: {len(vendas_produto)}")
    print("\n=== Prévia: Top 5 (visualização) ===")
    display(vendas_produto.head(5))

    # 6) Total geral e campetoo
    total_geral = vendas_produto['Total'].sum()
    total_fmt = f"{total_geral:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    print(f"\nTotal geral de vendas: R$ {total_fmt}")

    top = vendas_produto.iloc[0]
    top_valor = f"{top['Total']:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    print(f"\nProduto campeão: {top['Produto']} com total de R$ {top_valor}")

    return vendas_produto

# Executa agora
vendas_por_produto = analisar_vendas()




Produtos totais no arquivo: 30

=== Prévia: Top 5 (visualização) ===


,Produto,Total
0,Caixa Bluetooth,25147.72
1,Fone Pro,16815.82
2,Smartwatch,14902.39
3,Mouse Óptico,10154.66
4,Bola Oficial,8818.55



Total geral de vendas: R$ 187.183,68

Produto campeão: Caixa Bluetooth com total de R$ 25.147,72


In [10]:
from IPython.display import display

CLEAN_PATH = (ROOT / "data" / "processed" / "data_clean.csv").resolve()
_df = pd.read_csv(CLEAN_PATH, parse_dates=["Data"])
_df["Quantidade"] = pd.to_numeric(_df["Quantidade"], errors="coerce").fillna(0)
_df["Preco"] = pd.to_numeric(_df.get("Preco"), errors="coerce").fillna(0.0)
_df["Total"] = (_df["Quantidade"] * _df["Preco"]).astype(float)

_mensal_display = (
    _df.groupby(_df['Data'].dt.to_period('M'))['Total']
      .sum()
      .astype(float)
      .to_timestamp()
      .reset_index()
)
_mensal_display['Data'] = _mensal_display['Data'].dt.strftime('%Y-%m')
print('Faturamento mensal (R$):')
display(_mensal_display.rename(columns={'Total': 'Total_R$'}))

_top5_table = (
    _df.groupby('Produto', as_index=False)['Total']
      .sum()
      .sort_values('Total', ascending=False)
      .head(5)
)
print('Top-5 produtos (faturamento em R$):')
display(_top5_table.rename(columns={'Total': 'Total_R$'}))

_fevereiro_table = (
    _df[_df['Data'].dt.to_period('M') == pd.Period('2023-02', freq='M')]
      .groupby('Categoria', as_index=False)['Total']
      .sum()
      .sort_values('Total', ascending=False)
)
print('Categorias que mais venderam em fevereiro/2023:')
display(_fevereiro_table.rename(columns={'Total': 'Total_R$'}))


Faturamento mensal (R$):


,Data,Total_R$
0,2023-01,19706.82
1,2023-02,21617.00
2,2023-03,9707.27
3,2023-04,17590.81
4,2023-05,10663.76
5,2023-06,9000.85
6,2023-07,12752.57
7,2023-08,17844.73
8,2023-09,16476.35
9,2023-10,12907.68


Top-5 produtos (faturamento em R$):


,Produto,Total_R$
7,Caixa Bluetooth,25147.72
14,Fone Pro,16815.82
26,Smartwatch,14902.39
23,Mouse Óptico,10154.66
2,Bola Oficial,8818.55


Categorias que mais venderam em fevereiro/2023:


,Categoria,Total_R$
2,Eletrônicos,7630.34
3,Esporte,4962.71
0,Brinquedos,3532.05
1,Casa & Cozinha,2516.57
4,Material-Escolar,2267.70
5,Moda,707.63
